# OS
- ubuntu
- Intel i9 10900
- NVIDIA GeForce RTX 3080
- Memory 64GB

## Import

In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

import warnings
warnings.filterwarnings(action = 'ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor

## Data Load

In [21]:
train = pd.read_csv("../DAT/melt_data60.csv") # 모델 사용 형태에 맞게 정제된 데이터 불러오기

## Preprocessing

In [23]:
#필요하지 않은 칼럼 제거
train = train.drop(["제품", "대분류", "중분류", "소분류", "브랜드"], axis=1)

#label encoding 적용

lists = ["Weekday", "주말여부", "공휴일여부", "휴일여부"]

le = LabelEncoder()

for i in lists:
    train[i] = le.fit_transform(train[i])

In [24]:
#target 생성
train["COUNTS+1"] = train["COUNT"].shift(periods=-1) 

#train데이터와 test데이터셋 만들기
test = train[(train["Year"] == 2023) & (train["Month"] == 4) & (train["Day"] == 4)]
train = train[~((train["Year"] == 2023) & (train["Month"] == 4) & (train["Day"] == 4))]

In [27]:
#count_0 칼럼이랑 중복되는 칼럼이기에 제거

train = train.drop("COUNT", axis=1)
test = test.drop("COUNT", axis=1)

In [28]:
#train validation 구분

x_tr, x_val, y_tr, y_val = train_test_split(train.drop("COUNTS+1", axis=1), train["COUNTS+1"], test_size = 0.2, shuffle = True, random_state = 56 , stratify = train["ID"])

In [29]:
#test 데이터에는 필요없는 칼럼이므로 제거

test = test.drop("COUNTS+1", axis=1)

## Run!!

In [ ]:
weekend_list = [8,9,15,16,22,23] #주말여부와 휴일여부를 체크할 리스트
errors = []
answer = pd.DataFrame(columns = ['ID'] + [f'2023-04-{i+5:02d}' for i in range(21)]) #4월 5일부터 4월 25일까지의 칼럼 생성


for i in tqdm(range(len(train["ID"].unique()))) : #각 id별로 학습
    x_tr_ml = x_tr[x_tr["ID"] == i]
    x_val_ml = x_val[x_val["ID"] == i]
    y_tr_ml = y_tr[x_tr["ID"] == i]
    y_val_ml = y_val[x_val["ID"] == i]
    
    model = ExtraTreesRegressor(n_estimators = 200,
                         random_state = 41)
    model.fit(x_tr_ml, y_tr_ml)
    pred = model.predict(x_val_ml)
    error = mean_squared_error(pred, y_val_ml, squared=False)
    errors.append(error)
    
    answer.loc[i] = i
    
    test_ml = test[test["ID"] == i]
    
    #4월 5일의 판매량을 4월 4일부터 이전 60일데이터로 예측. 이후 4월 6일의 판매량을 예측했던 4월 5일 데이터와 이전 60일 데이터로 예측
    #이를 반복하여 총 21일치를 예측한다
    
    for j in range(21) : 
        ans = model.predict(test_ml)
        test_ml.iloc[0, 8:] = test_ml.iloc[0, 8:].shift(1)
        test_ml.iloc[0, 8] = ans
        
        test_ml.iloc[0, 3] = 5+j #예측에 사용한 day에 맞는 수로 바꿔주기
        
        if test_ml.iloc[0,3] in (weekend_list) : #주말 휴일
            test_ml.iloc[0,5] = 1
            test_ml.iloc[0,7] = 1
        else :
            test_ml.iloc[0,5] = 0
            test_ml.iloc[0,7] = 0
        
        test_ml.iloc[0, 4] = (test_ml.iloc[0,3] - 3) % 7 #weekand
        answer.iloc[i, j+1] = float(ans)
        

print(np.mean(errors))


## Save

In [ ]:
answer.to_csv("../OUT/ExtraResult.csv", index=False)